In [6]:
# Import the CARLA Python API library and some utils
import carla 
import math 
import random 
import time 
import json

client = carla.Client('ines-gpu-01.informatik.uni-mannheim.de', 2001)
client.load_world("Town10HD")
world = client.get_world()
map = world.get_map()
topology = map.get_topology()

topology_dict = {}

for x in topology:
    predkey = "R" + str(x[0].road_id) + "L" + str(x[0].lane_id)
    suckey = "R" + str(x[1].road_id) + "L" + str(x[1].lane_id)
    if not predkey in topology_dict:
        topology_dict[predkey] = []
    topology_dict[predkey].append(suckey)

with open("Town10HD_topology.json", "w") as f:
    json.dump(topology_dict, f)
